In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import os

home = '/content/drive/MyDrive/2ReadyForFineTuning/MultimodalQA/'
os.chdir(home)
imroot = home+'MultiModalQA_all_images'

In [4]:
prefix = 'Table_Passage_Image'

In [5]:
import json
import ast
datalen = 0
dumpdict = { "info": {}, "task_type": "Open-Ended", "data_type": "multimodalqa_table_passage_image", "license": {}, "data_subtype": prefix, "questions": [] }

with open(home+'Table+Image/'+prefix+'.jsonl') as f:
  data = f.readlines()
  print(len(data))
  global datalen
  datalen = len(data)
  for i in data[0:int(datalen*0.8)]:
    #print(i)
    ijson = json.loads(i)
    imlist = ijson['image_used']
    #print(imlist)
    anslist = ijson['answer_choices']
    try:
      #print(len(anslist))
      assert len(anslist) == 2
    except:
      print(i)
    dumpdict['questions'].append( { "image_id": imroot+"/"+imlist[0], "question": ijson['question']+" "+ijson['table_passage'], "question_id": ijson['qid'], "answer_choices": anslist, "answer_id": int(ijson['answers']) } )

with open('/content/'+prefix+'_train.jsonl','w+') as w:
  w.write(json.dumps(dumpdict,indent=4))

print(dumpdict)

2489
{'info': {}, 'task_type': 'Open-Ended', 'data_type': 'multimodalqa_table_passage_image', 'license': {}, 'data_subtype': 'Table_Passage_Image', 'questions': [{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/MultimodalQA/MultiModalQA_all_images/5ecb190d1382c18e7eb60e8fbab3cd17.jpg', 'question': 'Between the Queens Park Rangers or the team whose logo has a rose with leaves, which has higher points in the 1999-2000 Barnsley F.C. Season final league table? text Queens Park Rangers, links text Queens Park Rangers, wiki_title Queens Park Rangers F.C., url https//en.wikipedia.org/wiki/Queens_Park_Rangers_F.C.', 'question_id': '8d074ad059e688e64daf4839a8c08c3c', 'answer_choices': ['The team whose logo has a rose with leaves.', 'Queens Park Rangers'], 'answer_id': 1}, {'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/MultimodalQA/MultiModalQA_all_images/e94c7439cf99f31a54dc8f774bc0acbd.jpg', 'question': "Did Adam Kennedy or the player with a red stripe on his pants, red logo 

In [6]:
import json
f = open('/content/'+prefix+'_train.jsonl','r')
data_questions = json.load(f)
questions = data_questions['questions']

print(data_questions.keys())
print("Number of questions:", len(questions))

dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 1991


In [7]:
questions[0]

{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/MultimodalQA/MultiModalQA_all_images/5ecb190d1382c18e7eb60e8fbab3cd17.jpg',
 'question': 'Between the Queens Park Rangers or the team whose logo has a rose with leaves, which has higher points in the 1999-2000 Barnsley F.C. Season final league table? text Queens Park Rangers, links text Queens Park Rangers, wiki_title Queens Park Rangers F.C., url https//en.wikipedia.org/wiki/Queens_Park_Rangers_F.C.',
 'question_id': '8d074ad059e688e64daf4839a8c08c3c',
 'answer_choices': ['The team whose logo has a rose with leaves.',
  'Queens Park Rangers'],
 'answer_id': 1}

In [9]:
from transformers import ViltConfig

config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/136k [00:00<?, ?B/s]

In [10]:
import torch
from PIL import Image

class VQADataset(torch.utils.data.Dataset):

    def __init__(self, questions, processor):
        self.questions = questions
        self.processor = processor

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        questions = self.questions[idx]
        image = Image.open(questions['image_id']).convert("RGB")
        text = questions['question']
        encoding = self.processor(image, text, padding="max_length", truncation=True, return_tensors="pt")
        for k,v in encoding.items():
          encoding[k] = v.squeeze()
        targets = torch.zeros(len(questions['answer_choices']))
        targets[questions['answer_id']] = 1.0
        encoding["labels"] = targets
        return encoding

In [11]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
dataset = VQADataset(questions=questions,processor=processor)

In [13]:
dataset[0]

{'input_ids': tensor([  101,  2090,  1996,  8603,  2380,  7181,  2030,  1996,  2136,  3005,
         8154,  2038,  1037,  3123,  2007,  3727,  1010,  2029,  2038,  3020,
         2685,  1999,  1996,  2639,  1011,  2456, 25139,  1042,  1012,  1039,
         1012,  2161,  2345,  2223,  2795,  1029,  3793,  8603,  2380,   102]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'pixel_values': tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],

In [14]:
dataset[1].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])

In [15]:
len(dataset)

1991

In [16]:
processor.decode(dataset[0]['input_ids'])

'[CLS] between the queens park rangers or the team whose logo has a rose with leaves, which has higher points in the 1999 - 2000 barnsley f. c. season final league table? text queens park [SEP]'

In [17]:
labels = torch.nonzero(dataset[0]['labels']).squeeze().tolist()
labels

1

In [19]:
from transformers import ViltForQuestionAnswering

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm", num_labels=2)

model.to(device)

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-mlm and are newly initialized: ['classifier.0.bias', 'classifier.0.weight', 'classifier.1.bias', 'classifier.1.weight', 'classifier.3.bias', 'classifier.3.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

In [20]:
from torch.utils.data import DataLoader

def collate_fn(batch):
  input_ids = [item['input_ids'] for item in batch]
  pixel_values = [item['pixel_values'] for item in batch]
  attention_mask = [item['attention_mask'] for item in batch]
  token_type_ids = [item['token_type_ids'] for item in batch]
  labels = [item['labels'] for item in batch]

  encoding = processor.image_processor.pad(pixel_values, return_tensors="pt")

  batch = {}
  batch['input_ids'] = torch.stack(input_ids)
  batch['attention_mask'] = torch.stack(attention_mask)
  batch['token_type_ids'] = torch.stack(token_type_ids)
  batch['pixel_values'] = torch.stack(pixel_values)
  batch['pixel_mask'] = encoding['pixel_mask']
  batch['labels'] = torch.stack(labels)

  return batch

train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=1, shuffle=True)

In [21]:
batch = next(iter(train_dataloader))

In [22]:
for k,v in batch.items():
  print(k, v.shape)

input_ids torch.Size([1, 40])
attention_mask torch.Size([1, 40])
token_type_ids torch.Size([1, 40])
pixel_values torch.Size([1, 3, 480, 384])
pixel_mask torch.Size([1, 480, 384])
labels torch.Size([1, 2])


## Start Training

In [23]:
from tqdm.notebook import tqdm
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
# -- CHANGE BASED ON HOW MANY EPOCHS YOU WANT TO RUN FOR
e = 5

model.train()
for epoch in range(e):
   print(f"Epoch: {epoch}")
   for batch in tqdm(train_dataloader):
        batch = {k:v.to(device) for k,v in batch.items()}
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        #print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

Epoch: 0


  0%|          | 0/1991 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3368: DecompressionBombWarning: Image size (129967200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch: 1


  0%|          | 0/1991 [00:00<?, ?it/s]

...

  0%|          | 0/1991 [00:00<?, ?it/s]

Epoch: 3


  0%|          | 0/1991 [00:00<?, ?it/s]

Epoch: 4


  0%|          | 0/1991 [00:00<?, ?it/s]

## Inference

In [24]:
import json
import ast
datalen = 0
dumpdict = { "info": {}, "task_type": "Open-Ended", "data_type": "multimodalqa_table_passage_image", "license": {}, "data_subtype": prefix, "questions": [] }

with open(home+'Table+Image/'+prefix+'.jsonl') as f:
  data = f.readlines()
  print(len(data))
  global datalen
  datalen = len(data)
  for i in data[int(datalen*0.8):datalen]:
    #print(i)
    test_ijson = json.loads(i)
    test_imlist = test_ijson['image_used']
    #print(imlist)
    test_anslist = test_ijson['answer_choices']
    try:
      #print(len(anslist))
      assert len(test_anslist) == 2
    except:
      print(i)
    dumpdict['questions'].append( { "image_id": imroot+"/"+test_imlist[0], "question": test_ijson['question']+" "+test_ijson['table_passage'], "question_id": test_ijson['qid'], "answer_choices": test_anslist, "answer_id": int(test_ijson['answers']) } )

with open('/content/'+prefix+'_test.jsonl','w+') as w:
  w.write(json.dumps(dumpdict,indent=4))

print(dumpdict)

2489
{'info': {}, 'task_type': 'Open-Ended', 'data_type': 'multimodalqa_table_passage_image', 'license': {}, 'data_subtype': 'Table_Passage_Image', 'questions': [{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/MultimodalQA/MultiModalQA_all_images/faf04072bff0964b060d886e9408e2f2.jpg', 'question': 'what was the Players name, in assist table of 2007–08 Ligue 1 when the Club have a crown on top of their logo? text Nenê, links text Nenê, wiki_title Nenê (footballer, born 1981), url https//en.wikipedia.org/wiki/Nenê_(footballer,_born_1981)', 'question_id': '6e14555f45190749d2b37e07773712d8', 'answer_choices': ['NenÃª', 'Sidney Govou'], 'answer_id': 0}, {'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/MultimodalQA/MultiModalQA_all_images/5fc8fd3671a6a3c6ba4cd1824cd96e77.jpg', 'question': 'was Karim Benzema or the player name that has black spiked hair, the one with higher assist in ssists table of 2007–08 Ligue 1? text Samir Nasri, links text Samir Nasri, wiki_title Samir Na

In [25]:
import json
f = open('/content/'+prefix+'_test.jsonl','r')
test_data_questions = json.load(f)
test_questions = test_data_questions['questions']

print(test_data_questions.keys())
print("Number of questions:", len(test_questions))

dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 498


In [28]:
test_dataset = VQADataset(questions=test_questions,processor=processor)

In [29]:
len(test_dataset)

498

# Single Instance Inference

In [30]:
exid = 1
example = test_dataset[exid]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])


In [31]:
# add batch dimension + move to GPU
example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}

# forward pass
outputs = model(**example)

In [32]:
logits = outputs.logits
predicted_class = logits.argmax(-1).item()
print("Predicted answer:", predicted_class)
print("Ground-truth answer:", test_dataset[exid]['labels'].argmax(-1).item())

Predicted answer: 0
Ground-truth answer: 1


# Batch Inference

In [33]:
import csv
filprefix = prefix+"_result"

count = 0
st = 0
end = datalen-int(datalen*0.8)
with open(filprefix+'.csv', 'w+') as k:
  spamwriter = csv.writer(k)
  spamwriter.writerow(["qid","pred_ans","gt_ans","correctness"])
  for exid in range(st,end):
    correctness = 0
    example = test_dataset[exid]
    example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}
    outputs = model(**example)
    logits = outputs.logits
    predicted_class = logits.argmax(-1).item()
    gtclass = test_dataset[exid]['labels'].argmax(-1).item()
    if gtclass==predicted_class:
      count+=1
      correctness = 1
    spamwriter.writerow([exid,predicted_class, gtclass,correctness])